In [1]:
import pandas as pd
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0,1"
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings('ignore')

/home/adplz53/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [2]:
data_path = './dataset'
df = pd.read_pickle(data_path + '/text2ImgData.pkl')
num_training_sample = len(df)
n_images_train = num_training_sample
print('There are %d image in training data' % (n_images_train))

There are 7370 image in training data


In [3]:
#https://www.tensorflow.org/api_guides/python/image
#http://blog.csdn.net/mao_xiao_feng/article/details/75386477
#https://zhuanlan.zhihu.com/p/31160775
#https://stackoverflow.com/questions/41234822/append-to-series-in-python-pandas-not-working
#http://nbis.pixnet.net/blog/post/58150521-%5Bpython%5D-append%28%29%E8%88%87extend%28%29%E7%9A%84%E5%B7%AE%E5%88%A5
#https://stackoverflow.com/questions/10715965/add-one-row-in-a-pandas-dataframe

import scipy.misc

data_path = './dataset'

for i in range(1400,1450):#7370 純手動ˊ__>ˋ
   img_name = [data_path+df['ImagePath'].iloc[i]] #必須是list不是string
   print('Now is at:',i)
   df2 = pd.read_pickle('./testData_Augmentation.pkl')
    
   caption=df['Captions'].iloc[i]
   save_des=data_path+df['ImagePath'].iloc[i][0:23]

   img_reader = tf.WholeFileReader()
   filename_queue = tf.train.string_input_producer(img_name)
   _,image_jpg = img_reader.read(filename_queue)

   image_decode_jpeg = tf.image.decode_png(image_jpg)
   image_decode_jpeg = tf.image.convert_image_dtype(image_decode_jpeg, dtype=tf.float32)

   sess = tf.Session()
   coord = tf.train.Coordinator()
   threads = tf.train.start_queue_runners(sess=sess, coord=coord)

   #怕資料量太大 不用Resizing
   #Resizing
   resized_image1 = tf.image.resize_images(image_decode_jpeg, size=[500,600], method=tf.image.ResizeMethod.BILINEAR)
   resized_image2 = tf.image.resize_images(image_decode_jpeg, size=[250,320], method=tf.image.ResizeMethod.NEAREST_NEIGHBOR)
   resized_image3 = tf.image.resize_images(image_decode_jpeg, size=[300,300], method=tf.image.ResizeMethod.BICUBIC)
   resized_image4 = tf.image.resize_images(image_decode_jpeg, size=[1080,1920], method=tf.image.ResizeMethod.AREA)

   #Cropping
   resized_image5 = tf.image.resize_image_with_crop_or_pad(image_decode_jpeg, 500, 500) #局部放大
   resized_image6 = tf.image.resize_image_with_crop_or_pad(image_decode_jpeg, 300, 300) #外框加黑底
   resized_image7 = tf.image.central_crop(image_decode_jpeg,0.87) #局部放大
#    resized_image8 = tf.image.pad_to_bounding_box(image_decode_jpeg,0,0,200,200)
#    #圈出範圍(x1,y1,x2,y2)#不能太大有些圖會太小會失敗Q__Q
   resized_image8 = tf.image.resize_image_with_crop_or_pad(image_decode_jpeg,600,600)  #外框加黑底

   #Flipping, Rotating and Transposing
   resized_image9 = tf.image.flip_up_down(image_decode_jpeg)  #flip_up_down
   resized_image10 = tf.image.random_flip_up_down(image_decode_jpeg)  #random_flip_up_down
   resized_image11 = tf.image.flip_left_right(image_decode_jpeg)  #flip_left_right
   resized_image12 = tf.image.transpose_image(image_decode_jpeg)  #transpose

   #Image Adjustments
   resized_image13 = tf.image.adjust_brightness(image_decode_jpeg,0.003)  
   resized_image14 = tf.image.adjust_contrast(image_decode_jpeg,0.3)  
   resized_image15 = tf.image.random_hue(image_decode_jpeg,0.2)  
   resized_image16 = tf.image.per_image_standardization(image_decode_jpeg)  
   
   #output image
   scipy.misc.imsave(save_des+'_01.jpg', sess.run(resized_image1))
   scipy.misc.imsave(save_des+'_02.jpg', sess.run(resized_image2))
   scipy.misc.imsave(save_des+'_03.jpg', sess.run(resized_image3))
   scipy.misc.imsave(save_des+'_04.jpg', sess.run(resized_image4))
   scipy.misc.imsave(save_des+'_05.jpg', sess.run(resized_image5))
   scipy.misc.imsave(save_des+'_06.jpg', sess.run(resized_image6))
   scipy.misc.imsave(save_des+'_07.jpg', sess.run(resized_image7))
   scipy.misc.imsave(save_des+'_08.jpg', sess.run(resized_image8))
   scipy.misc.imsave(save_des+'_09.jpg', sess.run(resized_image9))
   scipy.misc.imsave(save_des+'_10.jpg', sess.run(resized_image10))
   scipy.misc.imsave(save_des+'_11.jpg', sess.run(resized_image11))
   scipy.misc.imsave(save_des+'_12.jpg', sess.run(resized_image12))
   scipy.misc.imsave(save_des+'_13.jpg', sess.run(resized_image13))
   scipy.misc.imsave(save_des+'_14.jpg', sess.run(resized_image14))
   scipy.misc.imsave(save_des+'_15.jpg', sess.run(resized_image15))
   scipy.misc.imsave(save_des+'_16.jpg', sess.run(resized_image16))

#    insert pickle
   df2=df2.append({'Captions':caption,'ImagePath':df['ImagePath'].iloc[i][0:23]+'_01.jpg'},ignore_index=True)
   df2=df2.append({'Captions':caption,'ImagePath':df['ImagePath'].iloc[i][0:23]+'_02.jpg'},ignore_index=True)
   df2=df2.append({'Captions':caption,'ImagePath':df['ImagePath'].iloc[i][0:23]+'_03.jpg'},ignore_index=True)
   df2=df2.append({'Captions':caption,'ImagePath':df['ImagePath'].iloc[i][0:23]+'_04.jpg'},ignore_index=True)
   df2=df2.append({'Captions':caption,'ImagePath':df['ImagePath'].iloc[i][0:23]+'_05.jpg'},ignore_index=True)
   df2=df2.append({'Captions':caption,'ImagePath':df['ImagePath'].iloc[i][0:23]+'_06.jpg'},ignore_index=True)
   df2=df2.append({'Captions':caption,'ImagePath':df['ImagePath'].iloc[i][0:23]+'_07.jpg'},ignore_index=True)
   df2=df2.append({'Captions':caption,'ImagePath':df['ImagePath'].iloc[i][0:23]+'_08.jpg'},ignore_index=True)
   df2=df2.append({'Captions':caption,'ImagePath':df['ImagePath'].iloc[i][0:23]+'_09.jpg'},ignore_index=True)
   df2=df2.append({'Captions':caption,'ImagePath':df['ImagePath'].iloc[i][0:23]+'_10.jpg'},ignore_index=True)
   df2=df2.append({'Captions':caption,'ImagePath':df['ImagePath'].iloc[i][0:23]+'_11.jpg'},ignore_index=True)
   df2=df2.append({'Captions':caption,'ImagePath':df['ImagePath'].iloc[i][0:23]+'_12.jpg'},ignore_index=True)
   df2=df2.append({'Captions':caption,'ImagePath':df['ImagePath'].iloc[i][0:23]+'_13.jpg'},ignore_index=True)
   df2=df2.append({'Captions':caption,'ImagePath':df['ImagePath'].iloc[i][0:23]+'_14.jpg'},ignore_index=True)
   df2=df2.append({'Captions':caption,'ImagePath':df['ImagePath'].iloc[i][0:23]+'_15.jpg'},ignore_index=True)
   df2=df2.append({'Captions':caption,'ImagePath':df['ImagePath'].iloc[i][0:23]+'_16.jpg'},ignore_index=True)
   
   df2.to_pickle('./testData_Augmentation.pkl')

# RuntimeError: can't start new thread =>太暴力了QAQ

Now is at: 1400
Now is at: 1401
Now is at: 1402
Now is at: 1403
Now is at: 1404
Now is at: 1405
Now is at: 1406
Now is at: 1407
Now is at: 1408
Now is at: 1409
Now is at: 1410
Now is at: 1411
Now is at: 1412
Now is at: 1413
Now is at: 1414
Now is at: 1415
Now is at: 1416
Now is at: 1417
Now is at: 1418
Now is at: 1419
Now is at: 1420
Now is at: 1421
Now is at: 1422
Now is at: 1423
Now is at: 1424
Now is at: 1425
Now is at: 1426
Now is at: 1427
Now is at: 1428
Now is at: 1429
Now is at: 1430
Now is at: 1431
Now is at: 1432
Now is at: 1433
Now is at: 1434
Now is at: 1435
Now is at: 1436
Now is at: 1437
Now is at: 1438
Now is at: 1439
Now is at: 1440
Now is at: 1441
Now is at: 1442
Now is at: 1443
Now is at: 1444
Now is at: 1445
Now is at: 1446
Now is at: 1447
Now is at: 1448
Now is at: 1449


In [4]:
df3 = pd.read_pickle('./testData_Augmentation.pkl')
df3

,Captions,ImagePath
0,"[[2430, 2428, 2431, 2427, 2436, 2432, 2450, 24...",/102flowers/image_08110.jpg
1,"[[2430, 2428, 2431, 2427, 2436, 2432, 2440, 24...",/102flowers/image_07749.jpg
2,"[[2435, 2428, 2505, 2431, 2444, 2427, 2433, 24...",/102flowers/image_04381.jpg
3,"[[2430, 2428, 2431, 2563, 2437, 2427, 2433, 24...",/102flowers/image_04518.jpg
4,"[[2435, 2428, 2427, 2432, 5409, 2429, 2432, 24...",/102flowers/image_07620.jpg
5,"[[2430, 2428, 2442, 2450, 2439, 2441, 2433, 24...",/102flowers/image_00724.jpg
6,"[[2428, 2433, 2438, 2427, 2429, 2487, 2440, 36...",/102flowers/image_00550.jpg
7,"[[2430, 2428, 2442, 2438, 2439, 2441, 2433, 24...",/102flowers/image_07209.jpg
8,"[[2428, 2431, 2427, 2436, 2432, 2440, 2433, 24...",/102flowers/image_02334.jpg
9,"[[2518, 2428, 2470, 2451, 2510, 2448, 2427, 24...",/102flowers/image_07389.jpg


In [5]:
Captions=df['Captions'].iloc[11]
Captions=df['Captions'][0][0]
ImagePath=df['ImagePath'][0]
print(Captions)
print(ImagePath)
print(df['ImagePath'][0][19:23])
print(df['ImagePath'][0][0:23])
#print(df['ImagePath'][91])
print(df['ImagePath'].iloc[91])
#https://stackoverflow.com/questions/31163495/error-when-accessing-pandas-dataframe-index

['2430', '2428', '2431', '2434', '2483', '2440', '2427', '2433', '2444', '2429', '2437', '2662', '6372', '6372', '6372', '6372', '6372', '6372', '6372', '6372']
/102flowers/image_05117.jpg
5117
/102flowers/image_05117
/102flowers/image_06808.jpg
